**SA367 &#x25aa; Mathematical Models for Decision Making &#x25aa; Spring 2022 &#x25aa; Uhan**

# Lesson 9. The Self-Sufficient Marine 

## The problem

__Self-sufficiency__ is the ability to carry out a mission without external support or aid, which is critical for operations in environments with limited infrastructure and logistical support.
Such scenarios are common in expeditionary missions for the Marines.
Perhaps the most fundamental question of self-sufficiency for these missions is whether a squad can be successful with the items that the Marines are able to carry.

Suppose you have a list of the items that a Marine may carry for a particular mission. 
In addition, suppose you have for each item:

- the __weight__ of the item, in units of 0.1 lbs

- the __value__ of carrying one unit of the item for the mission, on a scale from 0 to 10

- __lower and upper bounds__ on how many of the item needs to be carried

- whether the item is __partially shared__: that is, if the item can be shared among a few Marines if needed (e.g. binoculars) 

- whether the item is __shared__: that is, if an item should be used by an entire squad (e.g. metal detector)

Assume that a Marine can carry 65 lbs, at most 2 *types* of partially shared items, and 0 shared items.
The objective is to determine how many of each item a Marine should carry in order to maximize the total value of the items.

Formulate this problem as a dynamic program by giving its shortest/longest path representation.

*Note.* This problem is a simpler variant of the one studied in this paper:

> J. Simon, A. Apte, E. Reginier. An application of the multiple knapsack problem: the self-sufficient marine. *European Journal of Operational Research* 256: 868-876 (2017).

## Setting up the data

- In the same folder as this notebook, there is a CSV file named `hot-SOP.csv`, that contains the data described above
    - This data is a modified version of the data used in the paper by Simon, Apte, and Reginier (2017) cited above
    - The items are based on the Standard Operating Procedures policy for hot climate training for hiking 20 kilometers
    - The value assigned to each item was determined by analyzing the preferences of many individual Marines


- Let's take a look using Pandas


- First, let's import Pandas:

In [1]:
import pandas as pd

- Now we can read the CSV file into a Pandas DataFrame:

In [2]:
df = pd.read_csv('hot-SOP.csv')

- Let's peek at the first 5 rows of the DataFrame:

In [3]:
# Solution
df.head()

,Item,Weight,Value,Lower,Upper,Shared,PartialShared
0,Grenade pouches (2),3,7.1,0,2,0,0
1,Magazine pouches (3),3,7.1,0,6,0,0
2,BAT and HIIDE System,22,9.0,0,1,1,0
3,Batteries PRC-152,7,8.3,0,18,1,0
4,Binoculars/Spotting scope,39,4.7,0,1,0,1


- We see that whether an item is shared or not is indicated by a 1 or 0
    - Same thing goes for whether an item is partially shared or not


- We'll use the __index__ to refer to the different items:
    - e.g., item 2 is the BAT and HIIDE System


- Recall that we can convert DataFrame columns to dictionaries using `.to_dict()` 

- Let's create some dictionaries that will help us access the different pieces of information about each item: 

In [4]:
item = df['Item'].to_dict()
weight = df['Weight'].to_dict()
value = df['Value'].to_dict()
lower = df['Lower'].to_dict()
upper = df['Upper'].to_dict()
shared = df['Shared'].to_dict()
partial_shared = df['PartialShared'].to_dict()

- If this worked correctly, we should be able to get the weight and value of the BAT and HIIDE System as follows:

In [5]:
# Solution
print(f'Weight of BAT and HIIDE system: {weight[2]}')
print(f'Value of BAT and HIIDE system: {value[2]}')

Weight of BAT and HIIDE system: 22
Value of BAT and HIIDE system: 9.0


## Solving the DP

- There are three important constants in our problem: the number of items, the weight capacity, and the number of partially shared items allowed 

- Let's create variables to hold these constants

- This way, we can easily adapt our code to accommodate similar DPs

In [6]:
# Solution
T = len(item)
N1 = 650
N2 = 2

- Next, let's import `networkx` and `bellmanford`:

In [7]:
import networkx as nx
import bellmanford as bf

- As usual, we start with an empty digraph:

In [8]:
# Solution
G = nx.DiGraph()

- Next, let's add the stage-state nodes:

In [9]:
# Solution
for t in range(0, T + 1):
    for n1 in range(0, N1 + 1):
        for n2 in range(0, N2 + 1): 
            G.add_node((t, n1, n2))

- We can't forget the "end" node!

In [10]:
# Solution
G.add_node('end')

- Quick check: how many nodes do we have in our graph?

In [11]:
# Solution
print(G.number_of_nodes())

89839


- Now it's time to add the edges

- Let's start with the edges corresponding to the decision of how many of each item to carry:

In [12]:
# Add edges corresponding to how many of item t to carry
            
            # If item t is shared, don't consider it
            
            # Otherwise...
            
                # Let x be the number of item t to carry
                # x must be between the lower and upper bounds for item t
                    
                    # Weight capacity becomes... 
                    
                    # Partially shared item capacity becomes...
                    # Recall that we only care about the number of partially shared item *types* 
                    
                    # Add edge only if there's enough capacity
                    
                    # Don't forget we're maximizing the total value of the items carried!

In [13]:
# Solution
# Add edges corresponding to how many of item t to carry
for t in range(0, T):
    for n1 in range(0, N1 + 1):
        for n2 in range(0, N2 + 1):
            
            # If item t is shared, don't consider it
            if shared[t] == 1:
                G.add_edge((t, n1, n2), (t + 1, n1, n2), length=0)
            
            # Otherwise...
            else:
                # Let x be the number of item t to carry
                # x must be between the lower and upper bounds for item t
                for x in range(lower[t], upper[t] + 1):
                    
                    # Weight capacity becomes... 
                    m1 = n1 - weight[t] * x
                    
                    # Partially shared item capacity becomes...
                    # Recall that we only care about the number of partially shared item *types* 
                    if partial_shared[t] == 1 and x > 0:
                        m2 = n2 - 1
                    else:
                        m2 = n2
                    
                    # Add edge only if there's enough capacity
                    # Don't forget we're maximizing the total value of the items carried!
                    if m1 >= 0 and m2 >= 0:
                        G.add_edge((t, n1, n2), (t + 1, m1, m2), length=-value[t] * x)

- Now we can add the edges from the last stage to the end node

In [14]:
# Solution
for n1 in range(0, N1 + 1):
    for n2 in range(0, N2 + 1):
        G.add_edge((T, n1, n2), 'end', length=0) 

- Quick check: how many edges do we have in our graph?

In [15]:
# Solution
print(G.number_of_edges())

151197


- Finally, let's solve the shortest path problem we've constructed using the Bellman-Ford algorithm:

In [16]:
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, N1, N2), target='end', weight='length')

print(f'Negative cycle? {negative_cycle}')
print(f'Shortest path length: {length}')
print(f'Shortest path: {nodes}')

Negative cycle? False
Shortest path length: -307.9
Shortest path: [(0, 650, 2), (1, 644, 2), (2, 626, 2), (3, 626, 2), (4, 626, 2), (5, 626, 2), (6, 624, 2), (7, 593, 2), (8, 593, 2), (9, 386, 2), (10, 383, 1), (11, 381, 1), (12, 381, 1), (13, 243, 1), (14, 242, 1), (15, 239, 1), (16, 239, 1), (17, 236, 1), (18, 196, 1), (19, 193, 1), (20, 193, 1), (21, 158, 1), (22, 155, 1), (23, 155, 1), (24, 154, 1), (25, 145, 1), (26, 145, 1), (27, 145, 1), (28, 144, 1), (29, 144, 1), (30, 27, 1), (31, 26, 1), (32, 17, 1), (33, 17, 1), (34, 16, 1), (35, 16, 1), (36, 16, 1), (37, 16, 1), (38, 16, 1), (39, 16, 1), (40, 14, 1), (41, 13, 1), (42, 13, 1), (43, 11, 1), (44, 11, 1), (45, 9, 1), 'end']


- It's easy to see what the maximum total value of the items carried is... 

- However, how many of each item should we carry to get this maximum total value?

- Instead of reading through the path of 50-ish nodes to figure out how many of each item to carry, let's write some code to do this for us

- The $t$th node in the shortest path corresponds to stage/item $t$

- We can compute how many of item $t$ to carry as follows:

$$
\frac{(n_1 \mbox{ from stage } t) - (n_1 \mbox{ from stage } t + 1)}{\mbox{weight of item }  t} 
$$

- So...

In [17]:
# Go through each stage in the shortest path:
    
    # Node from stage t
    
    # Node from stage t + 1
    
    # Compute number of item t to carry
    
    # Print information

In [18]:
# Solution
# Go through each stage in the shortest path:
for i in range(0, T):
    
    # Node from stage t
    (t, n1, n2) = nodes[i]
    
    # Node from stage t + 1
    (next_t, next_n1, next_n2) = nodes[i + 1]
    
    # Compute number of item t to carry
    n_item = (n1 - next_n1) / weight[t]
    
    # Print information
    print(f'{item[t]:<40} {n_item}')

Grenade pouches (2)                      2.0
Magazine pouches (3)                     6.0
BAT and HIIDE System                     0.0
Batteries PRC-152                        0.0
Binoculars/Spotting scope                0.0
Boonie cover                             1.0
Boots                                    1.0
Breaching kit                            0.0
Camel back                               3.0
Cammie paint                             3.0
Canteen cup                              1.0
Combat tarp                              0.0
Covered canteen with one quart water     6.0
Dog tags                                 1.0
Dump pouch                               1.0
E-tool with carrier                      0.0
Eye protection/ear protection            1.0
First aid kit                            1.0
Gloves                                   1.0
Gortex top/bottom                        0.0
Helmet with cover                        1.0
Hip belt                                 1.0
Hygiene ge

## Incorporating shared items

- We ignored the possibility of carrying shared items in our model so far


- Suppose we can carry at most 2 types of partially shared items and at most 1 type of shared item


- How can we modify our dynamic program to accommodate this? Write a new dynamic program on paper

- Once you've figured out how you need to change our dynamic program, how can you modify the code above to solve the new dynamic program? 

In [19]:
# Solution
T = len(item)
N1 = 650
N2 = 2
N3 = 1

# Create empty digraph
G = nx.DiGraph()

# Add stage-stage nodes
for t in range(0, T + 1):
    for n1 in range(0, N1 + 1):
        for n2 in range(0, N2 + 1): 
            for n3 in range(0, N3 + 1):
                G.add_node((t, n1, n2))

# Add end node
G.add_node('end')

# Add edges
for t in range(0, T):
    for n1 in range(0, N1 + 1):
        for n2 in range(0, N2 + 1):
            for n3 in range(0, N3 + 1):
            
                # Let x be the number of item t to carry
                # x must be between the lower and upper bounds for item t
                for x in range(lower[t], upper[t] + 1):
                    # Weight capacity becomes... 
                    m1 = n1 - weight[t] * x

                    # Partially shared item capacity becomes...
                    if partial_shared[t] == 1 and x > 0:
                        m2 = n2 - 1
                    else:
                        m2 = n2
                        
                    # Shared item capacity becomes...
                    if shared[t] == 1 and x > 0:
                        m3 = n3 - 1
                    else:
                        m3 = n3

                    # Add edge only if there's enough capacity
                    if m1 >= 0 and m2 >= 0 and m3 >= 0:
                        G.add_edge((t, n1, n2, n3), (t + 1, m1, m2, m3), length=-value[t] * x)

# Add edges from last stage to end node
for n1 in range(0, N1 + 1):
    for n2 in range(0, N2 + 1):
        for n3 in range(0, N3 + 1):
            G.add_edge((T, n1, n2, n3), 'end', length=0) 

# Solve!
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, N1, N2, N3), target='end', weight='length')

print(f'Negative cycle? {negative_cycle}')
print(f'Shortest path length: {length}')
print(f'Shortest path: {nodes}')

# Print user-friendly output 
for i in range(0, T):
    (t, n1, n2, n3) = nodes[i]
    (next_t, next_n1, next_n2, next_n3) = nodes[i + 1]
    n_item = (n1 - next_n1) / weight[t]
    print(f'{item[t]:<40} {n_item}')

Negative cycle? False
Shortest path length: -332.79999999999995
Shortest path: [(0, 650, 2, 1), (1, 644, 2, 1), (2, 626, 2, 1), (3, 626, 2, 1), (4, 626, 2, 1), (5, 626, 2, 1), (6, 624, 2, 1), (7, 593, 2, 1), (8, 593, 2, 1), (9, 386, 2, 1), (10, 383, 1, 1), (11, 381, 1, 1), (12, 381, 1, 1), (13, 243, 1, 1), (14, 242, 1, 1), (15, 239, 1, 1), (16, 239, 1, 1), (17, 236, 1, 1), (18, 196, 1, 1), (19, 193, 1, 1), (20, 193, 1, 1), (21, 158, 1, 1), (22, 155, 1, 1), (23, 155, 1, 1), (24, 154, 1, 1), (25, 145, 1, 1), (26, 145, 1, 1), (27, 145, 1, 1), (28, 144, 1, 1), (29, 144, 1, 1), (30, 27, 1, 1), (31, 26, 1, 1), (32, 17, 1, 1), (33, 17, 1, 1), (34, 16, 1, 1), (35, 16, 1, 1), (36, 16, 1, 1), (37, 16, 1, 1), (38, 16, 1, 1), (39, 16, 1, 1), (40, 14, 1, 1), (41, 13, 1, 1), (42, 13, 1, 1), (43, 11, 1, 1), (44, 5, 1, 0), (45, 3, 1, 0), 'end']
Grenade pouches (2)                      2.0
Magazine pouches (3)                     6.0
BAT and HIIDE System                     0.0
Batteries PRC-152       

---

## Problems

### Problem 1 (Solving the airlift planning problem)

Solve the dynamic program we formulated in Example 1 of Lesson 8 using Python. What is the maximum possible total capability value? Which requirements should be put on the plane to achieve this maximum total capability value?

In [20]:
# Solution
# Import networkx and bellmanford
import networkx as nx
import bellmanford as bf

# Create variables for weight and volume capacity
MAX_WEIGHT = 80
MAX_VOLUME = 700 

# Create lists for capability values, weights, volumes
value = [50, 30, 80, 40, 70]
weight = [43, 17, 26, 4, 35]
volume = [250, 130, 370, 180, 400]

# Create empty graph
G = nx.DiGraph()

# Add stage-state nodes (t, n1, n2)
#   t = stage, consider requirement t
#   n1 = remaining weight
#   n2 = remaining volume
# 
# t = 0: large munitions
# t = 1: small munitions
# t = 2: food
# t = 3: medical supplies
# t = 4: repair parts
# t = 5: end of decision process
for t in range(0, 6):
    for n1 in range(0, MAX_WEIGHT + 1):
        for n2 in range(0, MAX_VOLUME + 1):
            G.add_node((t, n1, n2))

# Add end node
G.add_node("end")

# Add edges corresponding to decisions
# Remember we're maximizing, so the lengths should be negative of the values
for t in range(0, 5):
    for n1 in range(0, MAX_WEIGHT + 1):
        for n2 in range(0, MAX_VOLUME + 1):
            # Take requirement t, check if it fits first
            if n1 - weight[t] >= 0:
                if n2 - volume[t] >= 0:
                    G.add_edge((t, n1, n2), (t + 1, n1 - weight[t], n2 - volume[t]), 
                               length=-value[t])

            # Don't take requirement t
            G.add_edge((t, n1, n2), (t + 1, n1, n2), length=0)

# Add edges from last stage to end node
for n1 in range(0, MAX_WEIGHT + 1):
    for n2 in range(0, MAX_VOLUME + 1):
        G.add_edge((5, n1, n2), "end", length = 0)

# Solve the DP/shortest path problem using the Bellman-Ford algorithm
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, MAX_WEIGHT, MAX_VOLUME), 
                                                target="end", weight="length")

print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Negative cycle? False
Shortest path length: -150
Shortest path: [(0, 80, 700), (1, 80, 700), (2, 63, 570), (3, 37, 200), (4, 33, 20), (5, 33, 20), 'end']


_Write your notes here. Double-click to edit._

_Solution._

* The maximum possible total capability value is 150.

* To achieve this maximum total capability value, we should put the small munitions, food, and medical supplies on the C-17.